# システムプロンプト

In [ ]:
system_prompt = '''
You are a helpful assistant that helps people find information.Please ensure the output is always in Markdown format.
'''

# プロンプト

In [ ]:
prompt = '''
こんにちは
'''

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv


load_dotenv()  # take environment variables

# How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
DATABRICKS_TOKEN = os.environ['DATABRICKS_TOKEN']
# Alternatively in a Databricks notebook you can use this:
# DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken()[)
import httpx

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url=f"{os.environ['DATABRICKS_HOST']}{os.environ['OPTION_END_POINT_PARAM']}",
    timeout=httpx.Timeout(3600.0, read=3600.0, write=3600.0, connect=3600.0)
)

In [ ]:
system_prompt = system_prompt.replace("\\n","\n")
prompt = prompt.replace("\\n","\n") + "ただし、出力はMarkdownの形式で出力すること。"

In [ ]:
response = client.chat.completions.create(
    model=os.environ['DATABRICKS_MODEL_NAME'],
    messages=[
        {
            "role": "system",
            "content": f"{system_prompt}"
        },
        {
            "role": "user",
            "content": [ {
                "type": "text",
                "text": prompt
            }]
        }
    ]
)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")
print("Model:", response.model)
print("Usage:")
print("\tPrompt tokens:", response.usage.prompt_tokens)
print("\tCompletion tokens:", response.usage.completion_tokens)
print("\tTotal tokens:", response.usage.total_tokens)

In [ ]:
with open(f"output-anthropic-{os.environ['DATABRICKS_MODEL_NAME']}.markdown", "w") as o:
    print(response.choices[0].message.content, file=o)